In [1]:
import cv2
import torch
from utils.tool import LoadYaml
from module.detector import Detector

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
cfg = LoadYaml("configs/coco.yaml")  
model = Detector(cfg.category_num, True).to(device)
model.load_state_dict(torch.load("weights/weight_AP050.253207_280-epoch.pth", map_location=device))
model.eval()

Load yaml sucess...
load param...


Detector(
  (backbone): ShuffleNetV2(
    (first_conv): Sequential(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (stage2): Sequential(
      (0): ShuffleV2Block(
        (branch_main): Sequential(
          (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
          (4): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (6): BatchNorm2d(24, eps=1e-05, momentum=0.1, a

In [ ]:
# Initialize the camera
from utils.tool import handle_preds 
cap = cv2.VideoCapture(0)
assert cap.isOpened(), "The camera failed to open.！"

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Data preprocessing
        resized = cv2.resize(frame, (cfg.input_width, cfg.input_height))
        img_tensor = torch.from_numpy(resized.transpose(2, 0, 1)).float().unsqueeze(0) / 255.0
        img_tensor = img_tensor.to(device)
        
        preds = model(img_tensor)
        output = handle_preds(preds, device, 0.5)
        
        # Draw the detection box
        for box in output[0]:
            x1, y1 = int(box[0] * frame.shape[1]), int(box[1] * frame.shape[0])
            x2, y2 = int(box[2] * frame.shape[1]), int(box[3] * frame.shape[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Show the result
        cv2.imshow("FastestDet - Jupyter", frame)
        if cv2.waitKey(1) == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

/opt/anaconda3/lib/python3.12/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
